In [7]:
import requests
import time
import pandas as pd
from datetime import datetime, timedelta
import math

API_KEY = "73af62f1-882f-4c88-a477-ce6f38adde01"

def make_api_request(url):
    headers = {"api_key": API_KEY}
    
    while True:
        try:
            response = requests.get(url, headers=headers)
            response.raise_for_status()
            return response.json()
        except requests.HTTPError as e:
            if e.response.status_code == 429:
                retry_after = math.ceil(float(e.response.headers.get('Retry-After', '60')))
                print(f"Rate limit reached. Waiting for {retry_after} seconds before retrying...")
                time.sleep(retry_after)
            else:
                print(f"HTTP error occurred: {e}")
                return None
        except requests.RequestException as e:
            print(f"An error occurred: {e}")
            return None

def get_supported_exchanges():
    url = "https://api.coinalyze.net/v1/exchanges"
    return make_api_request(url)

def create_exchanges_dataframe(exchanges):
    return pd.DataFrame(exchanges)

def get_supported_future_markets():
    url = "https://api.coinalyze.net/v1/future-markets"
    return make_api_request(url)

def create_supported_futures_markets_dataframe(future_markets):
    df = pd.DataFrame(future_markets)
    
    # Convert expire_at from milliseconds to datetime
    df['expire_at'] = pd.to_datetime(df['expire_at'], unit='ms')
    
    return df

def filter_btc_perpetual_futures(futures_df):
    # Filter the DataFrame for BTC base asset, perpetual futures, and USD-based quote assets
    btc_perpetual_df = futures_df[
        (futures_df['base_asset'] == 'BTC') & 
        (futures_df['is_perpetual'] == True) &
        (futures_df['quote_asset'].isin(['USD', 'USDT', 'USDC']))
    ]
    
    return btc_perpetual_df

def get_open_interest_history(symbols, from_date, to_date, convert_to_usd=True):
    # Convert dates to UNIX timestamp
    from_timestamp = int(datetime.strptime(from_date, "%d-%m-%Y").timestamp())
    to_timestamp = int(datetime.strptime(to_date, "%d-%m-%Y").timestamp())
    
    # Split symbols into chunks of 10
    symbol_chunks = [symbols[i:i+10] for i in range(0, len(symbols), 10)]
    
    all_data = []
    
    for chunk in symbol_chunks:
        symbols_str = ",".join(chunk)
        url = f"https://api.coinalyze.net/v1/open-interest-history?symbols={symbols_str}&interval=daily&from={from_timestamp}&to={to_timestamp}&convert_to_usd={str(convert_to_usd).lower()}"
        
        response = make_api_request(url)
        if response:
            all_data.extend(response)
        
        # Sleep for 10 seconds between requests to avoid hitting rate limits
        time.sleep(10)
    
    # Process the data into the desired format
    processed_data = {}
    for item in all_data:
        symbol = item['symbol']
        for datapoint in item['history']:
            timestamp = pd.to_datetime(datapoint['t'], unit='s')
            if timestamp not in processed_data:
                processed_data[timestamp] = {}
            processed_data[timestamp][symbol] = datapoint['c']
    
    # Convert the processed data to a DataFrame
    df = pd.DataFrame.from_dict(processed_data, orient='index')
    df.index.name = 't'
    
    return df

def get_funding_rate_history(symbols, from_date, to_date, interval="daily"):
    # Convert dates to UNIX timestamp
    from_timestamp = int(datetime.strptime(from_date, "%d-%m-%Y").timestamp())
    to_timestamp = int(datetime.strptime(to_date, "%d-%m-%Y").timestamp())
    
    # Split symbols into chunks of 10
    symbol_chunks = [symbols[i:i+10] for i in range(0, len(symbols), 10)]
    
    all_data = []
    
    for chunk in symbol_chunks:
        symbols_str = ",".join(chunk)
        url = f"https://api.coinalyze.net/v1/funding-rate-history?symbols={symbols_str}&interval={interval}&from={from_timestamp}&to={to_timestamp}"
        
        response = make_api_request(url)
        if response:
            all_data.extend(response)
        
        # Sleep for 10 seconds between requests to avoid hitting rate limits
        time.sleep(10)
    
    # Process the data into the desired format
    processed_data = {}
    for item in all_data:
        symbol = item['symbol']
        for datapoint in item['history']:
            timestamp = pd.to_datetime(datapoint['t'], unit='s')
            if timestamp not in processed_data:
                processed_data[timestamp] = {}
            processed_data[timestamp][symbol] = datapoint['c']  # 'r' for rate instead of 'c' for close
    
    # Convert the processed data to a DataFrame
    df = pd.DataFrame.from_dict(processed_data, orient='index')
    df.index.name = 't'
    
    return df

def create_symbol_name_mapping(btc_perpetual_df, exchanges_df):
    print("Exchanges DataFrame columns:", exchanges_df.columns)
    print("Exchanges DataFrame first few rows:")
    print(exchanges_df.head())
    
    # Create a dictionary to map exchange codes to full names
    if 'code' in exchanges_df.columns and 'name' in exchanges_df.columns:
        exchange_name_map = dict(zip(exchanges_df['code'], exchanges_df['name']))
    else:
        print("Warning: 'code' or 'name' columns not found in exchanges DataFrame.")
        print("Columns available:", exchanges_df.columns)
        return {}  # Return an empty dictionary if we can't create the mapping
    
    # Create the symbol name mapping
    return {
        row['symbol']: f"{row['base_asset']}/{row['quote_asset']} - {exchange_name_map.get(row['exchange'], row['exchange'])}"
        for _, row in btc_perpetual_df.iterrows()
    }

def rename_columns_with_symbol_names(df, symbol_name_mapping):
    return df.rename(columns=symbol_name_mapping)

def main():
    # Fetch and process exchanges
    exchanges = get_supported_exchanges()
    if exchanges:
        exchanges_df = create_exchanges_dataframe(exchanges)
        exchanges_df.to_csv('supported_exchanges.csv', index=False)
        print("Exchanges DataFrame saved to 'supported_exchanges.csv'")
    else:
        print("Failed to retrieve supported exchanges.")
        return  # Exit if we can't get exchanges

    # Fetch and process future markets
    future_markets = get_supported_future_markets()
    if future_markets:
        futures_df = create_supported_futures_markets_dataframe(future_markets)
        futures_df.to_csv('supported_future_markets.csv', index=False)
        print("Future Markets DataFrame saved to 'supported_future_markets.csv'")
        
        # Filter for BTC perpetual futures with USD-based quote assets
        btc_perpetual_df = filter_btc_perpetual_futures(futures_df)
        btc_perpetual_df.to_csv('btc_perpetual_futures_usd.csv', index=False)
        print("BTC Perpetual Futures DataFrame saved to 'btc_perpetual_futures_usd.csv'")

        # Create symbol name mapping using both btc_perpetual_df and exchanges_df
        symbol_name_mapping = create_symbol_name_mapping(btc_perpetual_df, exchanges_df)

        if not symbol_name_mapping:
            print("Failed to create symbol name mapping. Check the exchanges data structure.")
            return

        # Load BTC perpetual futures symbols
        symbols = btc_perpetual_df['symbol'].tolist()

        # Get yesterday's date
        yesterday = (datetime.now() - timedelta(days=1)).strftime("%d-%m-%Y")

        # Fetch open interest history
        open_interest_df = get_open_interest_history(symbols, from_date="01-01-2022", to_date=yesterday)

        # Rename columns with symbol names
        open_interest_df = rename_columns_with_symbol_names(open_interest_df, symbol_name_mapping)

        open_interest_df.to_csv('open_interest_history.csv')
        print("Open Interest History DataFrame saved to 'open_interest_history.csv'")

        # Fetch funding rate history
        funding_rate_df = get_funding_rate_history(symbols, from_date="01-01-2022", to_date=yesterday)

        # Rename columns with symbol names
        funding_rate_df = rename_columns_with_symbol_names(funding_rate_df, symbol_name_mapping)

        # Save funding rate history to CSV
        funding_rate_df.to_csv('funding_rate_history.csv')
        print("Funding Rate History DataFrame saved to 'funding_rate_history.csv'")
    else:
        print("Failed to retrieve supported future markets.")

if __name__ == "__main__":
    main()

Exchanges DataFrame saved to 'supported_exchanges.csv'
Future Markets DataFrame saved to 'supported_future_markets.csv'
BTC Perpetual Futures DataFrame saved to 'btc_perpetual_futures_usd.csv'
Exchanges DataFrame columns: Index(['name', 'code'], dtype='object')
Exchanges DataFrame first few rows:
       name code
0  Poloniex    P
1    Vertex    V
2  Bitforex    D
3    Kraken    K
4   Bithumb    U
Open Interest History DataFrame saved to 'open_interest_history.csv'
Rate limit reached. Waiting for 15 seconds before retrying...
Funding Rate History DataFrame saved to 'funding_rate_history.csv'
